In [ ]:
import requests
import pandas as pd
from datetime import datetime

API_KEY = "b6472948a50b6381b5ec73208aa15809"
CITY = "Moscow"

url = f"http://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}&units=metric"
resp = requests.get(url)
data = resp.json()

df = pd.read_csv("temperature_data.csv", parse_dates=["timestamp"])
df = df.sort_values(["city", "timestamp"])

if resp.status_code == 200:
    current_temp = data['main']['temp']
    print(f"Текущая температура в {CITY}: {current_temp:.1f}°C")
else:
    print("Ошибка запроса:", data)

# is anomaly check
df_city = df[df["city"] == CITY].copy()

month = datetime.now().month
month_to_season = {12:"winter",1:"winter",2:"winter",
                   3:"spring",4:"spring",5:"spring",
                   6:"summer",7:"summer",8:"summer",
                   9:"autumn",10:"autumn",11:"autumn"}
current_season = month_to_season[month]

season_stats = df_city[df_city["season"] == current_season][["temperature"]].agg(["mean","std"])
mean_temp = season_stats.loc["mean", "temperature"]
std_temp = season_stats.loc["std", "temperature"]

if current_temp > mean_temp + 2*std_temp or current_temp < mean_temp - 2*std_temp:
    print("Температура аномальная")
else:
    print("Температура адекватная")

Текущая температура в Moscow: -1.4°C
Температура адекватная


## Попробуйте для получения текущей температуры использовать синхронные и асинхронные методы. Что здесь лучше использовать?

In [ ]:
import pandas as pd
import aiohttp
import asyncio
from datetime import datetime
import nest_asyncio

nest_asyncio.apply()

df = pd.read_csv("temperature_data.csv", parse_dates=["timestamp"])
df = df.sort_values(["city", "timestamp"])

API_KEY = "b6472948a50b6381b5ec73208aa15809"
cities = ["Berlin", "Cairo", "Moscow", "Beijing", "Dubai"]

month_to_season = {
    12:"winter",1:"winter",2:"winter",
    3:"spring",4:"spring",5:"spring",
    6:"summer",7:"summer",8:"summer",
    9:"autumn",10:"autumn",11:"autumn"
}
current_season = month_to_season[datetime.now().month]

async def fetch_temp(session, city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    async with session.get(url) as resp:
        data = await resp.json()
        if resp.status == 200:
            current_temp = data['main']['temp']
            
            #is anomaly check
            df_city = df[df["city"] == city]
            season_stats = df_city[df_city["season"] == current_season][["temperature"]].agg(["mean","std"])
            mean_temp = season_stats.loc["mean", "temperature"]
            std_temp = season_stats.loc["std", "temperature"]

            if current_temp > mean_temp + 2*std_temp or current_temp < mean_temp - 2*std_temp:
                status = "Аномальная"
            else:
                status = "Адекватная"

            return city, current_temp, status
        else:
            return city, None, "Ошибка"

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_temp(session, city) for city in cities]
        results = await asyncio.gather(*tasks)
        for city, temp, status in results:
            if temp is not None:
                print(f"{city}: {temp:.1f}°C — {status}")
            else:
                print(f"{city}:Ошибка")

await main()
